**Requirements**:

* QMCPy: `pip install qmcpy==2.1`
* LaTeX: `sudo apt update && sudo apt install -y texlive-full`
* testbook : `pip install testbook==0.4.2`
* Parsl: `pip install parsl==2025.7.28`

This notebook can be run interactively or in command line mode. To run in command line mode, use:
```bash
    jupyter nbconvert --to notebook --execute demos/talk_paper_demos/parsel_fest_2025/parsl_fest_2025.ipynb \
  --ExecutePreprocessor.kernel_name=qmcpy --ExecutePreprocessor.timeout=3600 --inplace
```

Our presentation slides for ParslFest are available at [Figma](https://www.figma.com/slides/k7EUosssNluMihkYTLuh1F/Parsl-Testbook-Speedup?node-id=174-95&t=t3jENVMltXWwdLdb-0).

In [1]:
try:
    import parsl as pl
except ModuleNotFoundError:
    !pip install -q parsl

In [2]:
import sys
import os
import time

# Ensure the path to the booktests directory is included (robust finder)
def _find_repo_root(start=os.getcwd()):
    cur = start
    while True:
        if os.path.exists(os.path.join(cur, 'pyproject.toml')):
            return cur
        parent = os.path.dirname(cur)
        if parent == cur:
            raise FileNotFoundError('repo root not found')
        cur = parent

sys.path.append(os.path.join(_find_repo_root(), 'test', 'booktests'))

# Configuration flags
# Set force_compute=False to reuse existing outputs; set True to force re-run
force_compute = True
is_debug = False

# Create output directory if it doesn't exist
print(f"{os.getcwd() = }")
output_dir = "output"
os.makedirs(output_dir, exist_ok=True)

# Clean local only files if force_compute is set
if force_compute:
    !(cd ../../.. && make clean_local_only_files)
    # remove output directory contents
    !rm -fr output/*.* runinfo/

os.getcwd() = '/workspaces/QMCSoftware/demos/talk_paper_demos/parsel_fest_2025'


## 1. Sequential Execution

In [ ]:
seq_fname = os.path.join(output_dir, "sequential_time.csv")
if (not os.path.exists(seq_fname)) or force_compute:
    start_time = time.time()
    if is_debug:
        !(cd ../../.. && make booktests_no_docker TESTS="tb_quickstart tb_qmcpy_intro tb_quickstart tb_qmcpy_intro")
    else:
        !(cd ../../.. && make booktests_no_docker )
    sequential_time = time.time() - start_time
    
    # save sequential time to a file in output_dir
    with open(seq_fname, "w") as f:
        f.write(f"{sequential_time:.2f}\n")

Checking notebook ↔ booktest coverage...
    Skipping demos/DAKOTA_Genz/dakota_genz.ipynb (requires large manual file / heavy memory use)


    Skipping demos/talk_paper_demos/Argonne_Talk_2023_May/Argonne_2023_Talk_Figures.ipynb (heavy LaTeX + many figures; skipped in booktests_no_docker)
    Skipping demos/talk_paper_demos/MCQMC2022_Article_Figures/MCQMC2022_Article_Figures.ipynb (MCQMC 2022 article figures; not run in CI)
    Skipping demos/talk_paper_demos/ProbFailureSorokinRao/prob_failure_gp_ci.ipynb (prob_failure_gp_ci talk demo; heavy GP / prob. failure example)
    Skipping demos/talk_paper_demos/Purdue_Talk_2023_March/Purdue_Talk_Figures.ipynb (Purdue 2023 talk figures; not a CI booktest target)
    Skipping demos/talk_paper_demos/parsel_fest_2025/01_sequential.ipynb (helper notebook for parsl_fest_2025; not a standalone booktest)
    Skipping demos/talk_paper_demos/parsel_fest_2025/02_parallel.ipynb (helper notebook for parsl_fest_2025; not a standalone booktest)
    Skipping demos/talk_paper_demos/parsel_fest_2025/03_visualize_speedup.ipynb (helper notebook for parsl_fest_2025; not a standalone booktest)
    Sk

In [ ]:
# free memory
import gc
gc.collect();

NameError: name 'seq_fname' is not defined